## Parse

In [33]:
from dataclasses import dataclass
import re
import json

RESULTS_FOLDER = "results/"

@dataclass
class SingleExperiment:
    protocol: str
    number_clients: int
    number_replicas: int
    payload: int

@dataclass
class Results:
    throughput: float
    read_latency_ms: float
    write_latency_ms: float
    overall_latency_ms: float


def protocol_name(paxos_variant: str) -> str:
    if paxos_variant == "single":
        return "Paxos"
    elif paxos_variant == "multi":
        return "Multi-Paxos"
    else:
        return paxos_variant

def process_results_file(exp_file: str) -> (SingleExperiment, Results):
    with open(f"{RESULTS_FOLDER}{exp_file}") as json_file:
        data = json.load(json_file)
        exp = SingleExperiment(protocol_name(data["config"]["paxos_variant"]),
                               data["config"]["clients"],
                               data["config"]["sm_replicas"],
                               data["config"]["payload"])

        results = results_from_output(data["outputs"][0])
        return exp, results


def results_from_output(output: str) -> Results:
    read_latency_ms = None
    write_latency_ms = None
    throughput = None
    lines = output.split("\n")
    for line in lines:
        if "[OVERALL], Throughput(ops/sec), " in line:
            throughput = float(re.findall(r"\d+\.\d+", line)[0])
        elif "[READ], AverageLatency(us), " in line:
            read_latency_ms = float(re.findall(r"\d+\.\d+", line)[0]) / 1000
        elif "[UPDATE], AverageLatency(us), " in line:
            write_latency_ms = float(re.findall(r"\d+\.\d+", line)[0]) / 1000
        elif "Op Timed out" in line:
            return None

    assert read_latency_ms is not None
    assert write_latency_ms is not None
    assert throughput is not None

    overall_latency_ms = (read_latency_ms + write_latency_ms) / 2
    return Results(throughput, read_latency_ms, write_latency_ms, overall_latency_ms)




In [34]:
import os

exp_results = []

for result_file in os.listdir(RESULTS_FOLDER):
    exp, results = process_results_file(result_file)
    if results is None:
        print("Exps that timed-out:")
        print(exp)
        continue
    exp_results.append((exp, results))
print(len(exp_results))


231


## Plot

In [35]:
import matplotlib.pyplot as plt

GRAPHS_FOLDER = "graphs/"
os.makedirs(GRAPHS_FOLDER, exist_ok=True)

@dataclass
class Line:
    label: str
    throughput_axis: list[float]
    latency_axis: list[float]


def group_results() -> dict[tuple[int, int], list[SingleExperiment, Results]]:
    configs = set(map(lambda x: (x[0].number_replicas, x[0].payload), exp_results))
    return {config: sorted([(exp, results) for (exp, results) in exp_results
                     if (exp.number_replicas, exp.payload) == config], key=lambda z: z[0].number_clients)
                     for config in configs}


def gen_throughput_latency_graph(config: tuple[int, int], lines: list[Line]):
    for line in lines:
        plt.plot(line.throughput_axis, line.latency_axis, marker="*", label=line.label)
    plt.xlabel("Throughput (1000 ops/sec)")
    plt.ylabel("Average Latency (ms)")
    plt.title(f"replicas={config[0]}, payload={config[1]}B")

    plt.tight_layout()
    plt.legend()

    plt.savefig(f"{GRAPHS_FOLDER}{config[0]}R_{config[1]}B.pdf")
    #plt.show()
    plt.clf()

In [36]:
from collections import defaultdict as dd


for config, lst_exp_results in group_results().items():
    lines = dd(lambda: Line("", [], []))
    for exp, res in lst_exp_results:
        lines[exp.protocol].label = exp.protocol
        lines[exp.protocol].throughput_axis.append(res.throughput / 1000)
        lines[exp.protocol].latency_axis.append(res.overall_latency_ms)

    gen_throughput_latency_graph(config, lines.values())

<Figure size 432x288 with 0 Axes>